In [2]:
# Importing Packages
import numpy as np 
import pandas as pd
import re
import json
import requests
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", 999)

import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
%matplotlib inline
plt.style.use("fivethirtyeight")

from sklearn.model_selection import train_test_split
import pickle

In [7]:
test = pd.read_csv('../data/testing.csv')
tt = test.drop(["ID"], axis=1)

In [4]:
url = 'https://openexchangerates.org/api/latest.json?app_id=c51b1508fb4145259b1c2fade72a2c04'
response = requests.get(url)
data = response.json()
rate = data['rates']['TWD']


In [8]:
tt.head()

,Unnamed: 0,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,23267,440000,2,2,1,39,0,0,0,0,0,0,58639,66698,43399,44534,25341,20000,30000,2000,24534,5000,20000,20839
1,15749,20000,2,2,1,23,1,2,0,0,0,2,15988,16219,17626,18398,19052,15429,794,1674,1446,1100,0,379
2,16429,50000,2,3,1,47,0,-1,-1,-1,-1,0,1091,390,1554,390,780,390,390,1554,390,780,0,390
3,6639,180000,2,3,2,27,1,-2,-2,-1,-1,-2,0,0,0,8081,0,0,0,0,8081,0,0,0
4,7129,210000,2,2,1,33,0,0,0,0,0,-1,8770,10972,22610,10161,5933,4164,6000,15042,5000,0,4164,816


In [9]:
data = [tt]
for d in data:    
    d.rename(columns={"PAY_0": "behind1", 
                        "PAY_2": "behind2", 
                        "PAY_3": "behind3", 
                        "PAY_4": "behind4",
                        "PAY_5": "behind5", 
                        "PAY_6": "behind6", 
                        "BILL_AMT1": "billed1", 
                        "BILL_AMT2": "billed2", 
                        "BILL_AMT3": "billed3", 
                        "BILL_AMT4": "billed4", 
                        "BILL_AMT5": "billed5", 
                        "BILL_AMT6": "billed6", 
                        "PAY_AMT1": "paid1", 
                        "PAY_AMT2": "paid2", 
                        "PAY_AMT3": "paid3", 
                        "PAY_AMT4": "paid4", 
                        "PAY_AMT5": "paid5", 
                        "PAY_AMT6": "paid6",
                        "SEX": "gender",
                        "EDUCATION": "education",
                        "MARRIAGE": "marriage",
                        "AGE": "age",
                        "LIMIT_BAL": "limit"}, inplace=True)
    d[['limit']] = d[['limit']]/rate
    d[['billed1', 'billed2', 'billed3', 'billed4', 'billed5', 'billed6']] = d[['billed1', 'billed2', 'billed3', 'billed4', 'billed5', 'billed6']].divide(rate, axis=1)
    d[['paid1', 'paid2', 'paid3', 'paid4', 'paid5', 'paid6']] = d[['paid1', 'paid2', 'paid3', 'paid4', 'paid5', 'paid6']].divide(rate, axis=1)
    d['limit'] = d['limit'].apply(lambda x: round(x, 2))
    d[['billed1', 'billed2', 'billed3', 'billed4', 'billed5', 'billed6']] = d[['billed1', 'billed2', 'billed3', 'billed4', 'billed5', 'billed6']].apply(lambda x: round(x, 2))
    d[['paid1', 'paid2', 'paid3', 'paid4', 'paid5', 'paid6']] = d[['paid1', 'paid2', 'paid3', 'paid4', 'paid5', 'paid6']].apply(lambda x: round(x, 2))
    d.replace({'marriage': {0:3}}, inplace=True)
    d.replace({'education': {5:4, 0:4, 6:4}}, inplace=True)

In [10]:
for d in data:
    d['avail6'] = (d.limit - d.billed6) / d.limit
    d['avail5'] = (d.limit - d.billed5) / d.limit
    d['avail4'] = (d.limit - d.billed4) / d.limit
    d['avail3'] = (d.limit - d.billed3) / d.limit
    d['avail2'] = (d.limit - d.billed2) / d.limit
    d['avail1'] = (d.limit - d.billed1) / d.limit
    d['avg_av'] = (d.avail1 + d.avail2 + d.avail3 + d.avail4 + d.avail5 + d.avail6) / 6

def delayed_payment(d):
    if (d.behind1 > 0) or (d.behind2 > 0) or (d.behind3 > 0) or (d.behind4 > 0) or (d.behind5 > 0) or (d.behind6 > 0):
        return 1
    else:
        return 0
for d in data:
    d['delayed'] = d.apply(delayed_payment, axis=1)

def total_months_with_delayed_payments(d):
    count = 0
    if (d.behind1 > 0):
        count += 1
    if (d.behind2 > 0):
        count += 1
    if (d.behind3 > 0):
        count += 1
    if (d.behind4 > 0):
        count += 1
    if (d.behind5 > 0):
        count += 1
    if (d.behind6 > 0):
        count += 1
    return count
for d in data:
    d['latemths'] = d.apply(total_months_with_delayed_payments, axis=1)


In [13]:
tt = tt.drop(["Unnamed: 0"], axis=1)

In [14]:
tt.head()

,limit,gender,education,marriage,age,behind1,behind2,behind3,behind4,behind5,behind6,billed1,billed2,billed3,billed4,billed5,billed6,paid1,paid2,paid3,paid4,paid5,paid6,avail6,avail5,avail4,avail3,avail2,avail1,avg_av,delayed,latemths
0,15754.29,2,2,1,39,0,0,0,0,0,0,2099.58,2388.14,1553.91,1594.55,907.34,716.10,1074.16,71.61,878.44,179.03,716.10,746.14,0.954546,0.942407,0.898786,0.901366,0.848413,0.866730,0.902041,0,0
1,716.10,2,2,1,23,1,2,0,0,0,2,572.45,580.72,631.10,658.74,682.16,552.44,28.43,59.94,51.77,39.39,0.00,13.57,0.228543,0.047396,0.080101,0.118699,0.189052,0.200600,0.144065,1,3
2,1790.26,2,3,1,47,0,-1,-1,-1,-1,0,39.06,13.96,55.64,13.96,27.93,13.96,13.96,55.64,13.96,27.93,0.00,13.96,0.992202,0.984399,0.992202,0.968921,0.992202,0.978182,0.984685,0,0
3,6444.94,2,3,2,27,1,-2,-2,-1,-1,-2,0.00,0.00,0.00,289.34,0.00,0.00,0.00,0.00,289.34,0.00,0.00,0.00,1.000000,1.000000,0.955106,1.000000,1.000000,1.000000,0.992518,1,1
4,7519.09,2,2,1,33,0,0,0,0,0,-1,314.01,392.85,809.56,363.82,212.43,149.09,214.83,538.58,179.03,0.00,149.09,29.22,0.980172,0.971748,0.951614,0.892333,0.947753,0.958238,0.950310,0,0


In [15]:
X_tt = tt[['limit', 'behind1', 'paid2', 'delayed', 'latemths', 'age', 'behind2', 'billed1', 'avg_av', 'avail1']]

In [16]:
X_tt.head()

,limit,behind1,paid2,delayed,latemths,age,behind2,billed1,avg_av,avail1
0,15754.29,0,71.61,0,0,39,0,2099.58,0.902041,0.866730
1,716.10,1,59.94,1,3,23,2,572.45,0.144065,0.200600
2,1790.26,0,55.64,0,0,47,-1,39.06,0.984685,0.978182
3,6444.94,1,0.00,1,1,27,-2,0.00,0.992518,1.000000
4,7519.09,0,538.58,0,0,33,0,314.01,0.950310,0.958238


In [17]:
pickle_in = open("../data/best_model.pickle","rb")
model = pickle.load(pickle_in)

In [18]:
y_pred_tt = model.predict(X_tt)

ValueError: Number of features of the model must match the input. Model n_features is 40 and input n_features is 10 